In [ ]:
!git clone https://github.com/NVIDIA/NeMo-Curator.git
!pip install --extra-index-url https://pypi.nvidia.com "./NeMo-Curator[all]"

Cloning into 'NeMo-Curator'...
remote: Enumerating objects: 3201, done.
remote: Counting objects: 100% (1136/1136), done.
remote: Compressing objects: 100% (560/560), done.
remote: Total 3201 (delta 860), reused 591 (delta 576), pack-reused 2065 (from 3)
Receiving objects: 100% (3201/3201), 5.83 MiB | 29.00 MiB/s, done.
Resolving deltas: 100% (1943/1943), done.
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Processing ./NeMo-Curator
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from nemo_curator import Modify
from nemo_curator.modifiers import UnicodeReformatter
from nemo_curator.utils.distributed_utils import read_data, write_to_disk
from nemo_curator.utils.file_utils import get_all_files_paths_under
from nemo_curator.datasets import DocumentDataset

In [ ]:
from huggingface_hub import login

login(
  token="", # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
from datasets import load_dataset
import pandas as pd
import os

# Load dataset from Hugging Face (replace 'dataset_name' with your actual dataset)
dataset = load_dataset('akashmaggon/jejuandneworleans_news_dataset')
dataset = dataset.rename_column("enhanced_text", "text")


In [ ]:
import random
from datasets import Dataset

# Assuming 'dataset' is your Hugging Face dataset
# Get the number of rows in the dataset
num_rows = len(dataset["train"])

# Randomly select a row index to duplicate
random_index = random.randint(0, num_rows - 1)

# Get the row to duplicate
row_to_duplicate = dataset["train"][random_index]

# Add the duplicated row to the dataset
dataset["train"] = Dataset.from_dict({
    key: dataset["train"][key] + [row_to_duplicate[key]]
    for key in dataset["train"].column_names
})

# Verify the result
print(f"Duplicated row at index {random_index}")
print(dataset["train"])

Duplicated row at index 31
Dataset({
    features: ['original_text', 'text', 'instruction', 'filename'],
    num_rows: 98
})


In [ ]:
row_to_duplicate

{'original_text': '"Houston").[[4]](https://en.wikipedia.org/wiki/<#cite_note-:2-4>) Although the FBI said Jabbar\'s choice of [Bourbon Street](https://en.wikipedia.org/wiki/</wiki/Bourbon_Street> "Bourbon Street") for his attack was unclear, it stated that he was inspired by ISIS and that it was still investigating his motives.[[54]](https://en.wikipedia.org/wiki/<#cite_note-:3-54>)',
 'text': "The FBI stated that while Jabbar's choice of Bourbon Street for his attack was unclear, he was inspired by ISIS. The investigation into his motives is ongoing.",
 'instruction': "What was the inspiration behind Jabbar's attack on Bourbon Street?",
 'filename': '2025_New_Orleans_truck_attack.txt'}

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['original_text', 'text', 'instruction', 'filename'],
        num_rows: 98
    })
})

In [ ]:
dataset["train"] = dataset["train"].remove_columns([col for col in dataset["train"].column_names if col != "text"])
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 98
    })
})

In [ ]:
input_dir_nemo = "nemo_input"
output_dir_nemo = "nemo_output"
main_dataset = "/main_dataset.parquet"

In [ ]:
# Convert to pandas DataFrame
df = dataset['train'].to_pandas()

# Create output directory if it doesn't exist
os.makedirs(input_dir_nemo, exist_ok=True)

# Save as parquet
df.to_parquet(input_dir_nemo + "/main_dataset.parquet")

In [ ]:
import os

# Define paths for Unicode formatted data
unicode_formatted_output_path = os.path.join(output_dir_nemo, "unicode_removed")

# Load the raw data
def load_dataset(input_data_dir, file_type="parquet"):
    print(input_data_dir)
    files = list(get_all_files_paths_under(input_data_dir))
    print(files)
    raw_data = read_data(files, file_type=file_type, backend="pandas", add_filename=True)
    dataset = DocumentDataset(raw_data)

    return dataset

raw_data = load_dataset(input_dir_nemo , file_type="parquet")

# # Initialize the Unicode reformatter
cleaner = Modify(UnicodeReformatter())

# Apply Unicode reformatting
cleaned_data = cleaner(raw_data)

# Save the cleaned data to disk
write_to_disk(cleaned_data.df, unicode_formatted_output_path, write_to_filename=True, output_type="parquet")

nemo_input
['nemo_input/main_dataset.parquet']
Reading 1 files with blocksize=None / files_per_partition=1
Writing to disk complete for 1 partition(s)


In [ ]:
from datasets import load_dataset

# Load the Parquet file as a Hugging Face dataset
parquet_dataset = load_dataset("parquet", data_files=unicode_formatted_output_path + main_dataset)

# Access the "train" split of the dataset
train_split = parquet_dataset["train"]

# Visualize the dataset (e.g., first few rows)
print(train_split)

# View the values in the "text" column
print(train_split[:10]["text"])  # Show the first 10 values in the 'text' key

# Optional: Use the Dataset's `to_pandas` method for further visualization
cleaned_dff = train_split.to_pandas()
print(cleaned_dff.head())


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 98
})
['The New Orleans truck attack occurred on January 1, 2025, at 3:16 a.m. CST (UTC-6) on Bourbon Street, New Orleans, Louisiana, U.S. The attack targeted pedestrians and involved a vehicle-ramming attack, a shootout, and mass murder. The weapons used in the attack were a Ford F-150 Lightning and a .308 AR-10 semi-automatic rifle.', 'The New Orleans truck attack resulted in 15 deaths, including the suspect, and left 35 people injured. The motive behind the attack is currently under investigation.', 'The 2025 New Orleans truck attack took place on January 1, 2025, at approximately 3:15 a.m. CST (UTC–6). A man drove a pickup truck into a crowd on Bourbon Street in New Orleans, Louisiana, United States. After crashing the truck, he exited the vehicle and engaged in a shootout with police officers before being fatally shot. The incident resulted in the deaths of fifteen people, including the suspect, and left at least 35 others injured. A

In [ ]:
df

,text
0,The New Orleans truck attack occurred on Janua...
1,The New Orleans truck attack resulted in 15 de...
2,The 2025 New Orleans truck attack took place o...
3,The attack took place during New Year celebrat...
4,The Federal Bureau of Investigation (FBI) iden...
...,...
93,"Following the crash, Jeju Air recorded an incr..."
94,"On 30 December, authorities ordered the inspec..."
95,The United States deployed investigators from ...
96,"GE Aerospace, which produced the engines on th..."


In [ ]:
cleaned_dff

,text
0,The New Orleans truck attack occurred on Janua...
1,The New Orleans truck attack resulted in 15 de...
2,The 2025 New Orleans truck attack took place o...
3,The attack took place during New Year celebrat...
4,The Federal Bureau of Investigation (FBI) iden...
...,...
93,"Following the crash, Jeju Air recorded an incr..."
94,"On 30 December, authorities ordered the inspec..."
95,The United States deployed investigators from ...
96,"GE Aerospace, which produced the engines on th..."


In [ ]:
from nemo_curator import AddId

# Define paths for input data and output with added IDs
add_id_input_data_dir = unicode_formatted_output_path
added_id_output_path = os.path.join(output_dir_nemo, "add_id")
add_ID_id_prefix = "VI_"

# Load the formatted dataset
dataset = DocumentDataset.read_parquet(add_id_input_data_dir + main_dataset)

# Initialize the AddId class with a specified prefix and start index
add_id = AddId(id_field="id", id_prefix=add_ID_id_prefix, start_index=0)

# Apply the ID addition to the dataset
id_dataset = add_id(dataset)

# Save the dataset with added IDs to disk
write_to_disk(id_dataset.df, output_path=added_id_output_path, write_to_filename=False, output_type="parquet")
# write_to_disk(id_dataset.df, output_path="outputssss", write_to_filename=False, output_type="parquet")

Reading 1 files with blocksize='1gb' / files_per_partition=None
Writing to disk complete for 1 partition(s)


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:441: UserWarning: If underlying Parquet data does not have a consistent column order, reading with blocksize might fail. Please use files_per_partition approach instead.
  warnings.warn(


In [ ]:
## Lets read in the added file with ID's

In [ ]:
from datasets import load_dataset

# Load the Parquet file as a Hugging Face dataset
parquet_dataset = load_dataset("parquet", data_files=added_id_output_path + "/part.0.parquet")

# Access the "train" split of the dataset
train_split = parquet_dataset["train"]

# Visualize the dataset (e.g., first few rows)
print(train_split)

# View the values in the "text" column
print(train_split[:10]["text"])  # Show the first 10 values in the 'text' key

# Optional: Use the Dataset's `to_pandas` method for further visualization
cleaned_dff = train_split.to_pandas()
print(cleaned_dff.head())


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'id'],
    num_rows: 98
})
['The New Orleans truck attack occurred on January 1, 2025, at 3:16 a.m. CST (UTC-6) on Bourbon Street, New Orleans, Louisiana, U.S. The attack targeted pedestrians and involved a vehicle-ramming attack, a shootout, and mass murder. The weapons used in the attack were a Ford F-150 Lightning and a .308 AR-10 semi-automatic rifle.', 'The New Orleans truck attack resulted in 15 deaths, including the suspect, and left 35 people injured. The motive behind the attack is currently under investigation.', 'The 2025 New Orleans truck attack took place on January 1, 2025, at approximately 3:15 a.m. CST (UTC–6). A man drove a pickup truck into a crowd on Bourbon Street in New Orleans, Louisiana, United States. After crashing the truck, he exited the vehicle and engaged in a shootout with police officers before being fatally shot. The incident resulted in the deaths of fifteen people, including the suspect, and left at least 35 others inju

In [ ]:
cleaned_dff

,text,id
0,The New Orleans truck attack occurred on Janua...,VI_-0000000000
1,The New Orleans truck attack resulted in 15 de...,VI_-0000000001
2,The 2025 New Orleans truck attack took place o...,VI_-0000000002
3,The attack took place during New Year celebrat...,VI_-0000000003
4,The Federal Bureau of Investigation (FBI) iden...,VI_-0000000004
...,...,...
93,"Following the crash, Jeju Air recorded an incr...",VI_-0000000093
94,"On 30 December, authorities ordered the inspec...",VI_-0000000094
95,The United States deployed investigators from ...,VI_-0000000095
96,"GE Aerospace, which produced the engines on th...",VI_-0000000096


In [ ]:
import os
from nemo_curator.modules import ExactDuplicates
from nemo_curator.datasets import DocumentDataset

# Define input and output paths
exact_dedup_input_dataset_dir = added_id_output_path
exact_dedup_base_output_path = os.path.join(output_dir_nemo, "exact_dedup")
exact_dedup_log_dir = os.path.join(exact_dedup_base_output_path, "log")
exact_dedup_output_dir = os.path.join(exact_dedup_base_output_path, "data")
deduped_output_dir = os.path.join(output_dir_nemo, "remove_duplicate")

# Create directories for logs and output
!mkdir -p {exact_dedup_log_dir}
!mkdir -p {exact_dedup_output_dir}
!mkdir -p {deduped_output_dir}

In [ ]:
from nemo_curator.utils.distributed_utils import get_client

def pre_imports():
    import cudf

client = get_client(cluster_type="gpu", set_torch_to_use_rmm=False)
client.run(pre_imports)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/scheduler-a9bqhq__', purging
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:44199
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:32883'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:34927', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34927
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:35894
INFO:distributed.scheduler:Receive client connection: Client-7918afa2-cbaf-11ef-9569-0242ac1c000c
INFO:distributed.core:Starting 

cuDF Spilling is enabled


{'tcp://127.0.0.1:34927': None}

In [ ]:
# Parameters for ExactDuplicates
exact_dedup_dataset_id_field = "id"
exact_dedup_dataset_text_field = "text"

# Load the input dataset
input_dataset = DocumentDataset.read_parquet(exact_dedup_input_dataset_dir + "/part.0.parquet", backend="cudf")

Reading 1 files with blocksize='1gb' / files_per_partition=None


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:436: UserWarning: If underlying Parquet data does not have consistent schema, reading with blocksize will fail. Please update underlying RAPIDS package to version 25.02 or higher, or use files_per_partition approach instead.
  warnings.warn(


In [ ]:
# Initialize and run exact deduplication
exact_dup = ExactDuplicates(
    logger=exact_dedup_log_dir,
    id_field=exact_dedup_dataset_id_field,
    text_field=exact_dedup_dataset_text_field,
    hash_method="md5",
    cache_dir=exact_dedup_output_dir
)
duplicates = exact_dup(dataset=input_dataset)

print(f"Number of exact duplicate files: {len(duplicates)}")

INFO:ExactDuplicates:Starting lazy hash generation
INFO:ExactDuplicates:Lazy hash generation complete for 1 partitions
INFO:ExactDuplicates:Starting execution for ExactDedup
/usr/local/lib/python3.10/dist-packages/nemo_curator/modules/exact_dedup.py:158: UserWarning: Output path fnemo_output/exact_dedup/data/_exact_duplicates.parquet already exists and will be overwritten
  warnings.warn(
INFO:ExactDuplicates:Time taken for Exact Dedup Computation = 8.145959615707397s and output written at nemo_output/exact_dedup/data/_exact_duplicates.parquet


Number of exact duplicate files: 2


In [ ]:
# Load the dataset and exact duplicates to identify and remove duplicate IDs
input_dataset = DocumentDataset.read_parquet(added_id_output_path, backend="cudf")
exact_duplicates = DocumentDataset.read_parquet(
    os.path.join(exact_dedup_output_dir, "_exact_duplicates.parquet"), backend="cudf"
)

# Extract list of duplicate document IDs
exact_docs_to_remove = exact_duplicates.df.map_partitions(
    lambda x: x[x._hashes.duplicated(keep="first")]
)

# Remove duplicated documents from the input dataset
result = input_dataset.df[
    ~input_dataset.df[exact_dedup_dataset_id_field].isin(exact_docs_to_remove[exact_dedup_dataset_id_field].compute())
]

# Save the final deduplicated dataset
write_to_disk(result, output_path=deduped_output_dir, write_to_filename=False, output_type="parquet")

Reading 1 files with blocksize='1gb' / files_per_partition=None
Reading 1 files with blocksize='1gb' / files_per_partition=None


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:436: UserWarning: If underlying Parquet data does not have consistent schema, reading with blocksize will fail. Please update underlying RAPIDS package to version 25.02 or higher, or use files_per_partition approach instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:436: UserWarning: If underlying Parquet data does not have consistent schema, reading with blocksize will fail. Please update underlying RAPIDS package to version 25.02 or higher, or use files_per_partition approach instead.
  warnings.warn(


Writing to disk complete for 1 partition(s)


In [ ]:
from datasets import load_dataset

# Load the Parquet file as a Hugging Face dataset
parquet_dataset = load_dataset("parquet", data_files=deduped_output_dir + "/part.0.parquet")

# Access the "train" split of the dataset
train_split = parquet_dataset["train"]

# Visualize the dataset (e.g., first few rows)
print(train_split)

# View the values in the "text" column
print(train_split[:10]["text"])  # Show the first 10 values in the 'text' key

# Optional: Use the Dataset's `to_pandas` method for further visualization
cleaned_dff = train_split.to_pandas()
print(cleaned_dff.head())


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 97
})
['The New Orleans truck attack occurred on January 1, 2025, at 3:16 a.m. CST (UTC-6) on Bourbon Street, New Orleans, Louisiana, U.S. The attack targeted pedestrians and involved a vehicle-ramming attack, a shootout, and mass murder. The weapons used in the attack were a Ford F-150 Lightning and a .308 AR-10 semi-automatic rifle.', 'The New Orleans truck attack resulted in 15 deaths, including the suspect, and left 35 people injured. The motive behind the attack is currently under investigation.', 'The 2025 New Orleans truck attack took place on January 1, 2025, at approximately 3:15 a.m. CST (UTC–6). A man drove a pickup truck into a crowd on Bourbon Street in New Orleans, Louisiana, United States. After crashing the truck, he exited the vehicle and engaged in a shootout with police officers before being fatally shot. The incident resulted in the deaths of fifteen people, including the suspect, and left at least 35 others inju

In [ ]:
cleaned_dff

,id,text
0,VI_-0000000000,The New Orleans truck attack occurred on Janua...
1,VI_-0000000001,The New Orleans truck attack resulted in 15 de...
2,VI_-0000000002,The 2025 New Orleans truck attack took place o...
3,VI_-0000000003,The attack took place during New Year celebrat...
4,VI_-0000000004,The Federal Bureau of Investigation (FBI) iden...
...,...,...
92,VI_-0000000092,"On 2 January, the Jeonnam Provincial Police Ag..."
93,VI_-0000000093,"Following the crash, Jeju Air recorded an incr..."
94,VI_-0000000094,"On 30 December, authorities ordered the inspec..."
95,VI_-0000000095,The United States deployed investigators from ...


In [ ]:
from nemo_curator.utils.config_utils import build_filter_pipeline
import warnings

# Define paths for input data and output data after heuristic filtering
HF_input_data_dir = deduped_output_dir
HF_output_path = os.path.join(output_dir_nemo, "heuristic_filtering")

# Create a directory for the configuration file if it doesn't exist
os.makedirs("config", exist_ok=True)
# Download the YAML configuration file for heuristic filtering
!wget https://raw.githubusercontent.com/NVIDIA/NeMo-Curator/main/config/heuristic_filter_en.yaml -O ./config/heuristic_filter_en.yaml

# Specify the path to the configuration file
filter_config_file = "./config/heuristic_filter_en.yaml"
os.makedirs(HF_output_path, exist_ok=True)

# Load the filters from the YAML configuration file
filter_pipeline = build_filter_pipeline(filter_config_file)

print("=========")
print(HF_input_data_dir + "/part.0.parquet")
print("=========")

# Load the dataset
dataset = DocumentDataset.read_parquet(HF_input_data_dir,  backend="pandas")

# Suppress specific warnings during filtering
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)
    # Apply the heuristic filters to the dataset
    result_data = filter_pipeline(dataset)
    print("==============")
    print(result_data)
    print("==============")

    # Save the filtered dataset to disk
    result_data.to_parquet(HF_output_path, write_to_filename=False)

--2025-01-05 21:53:27--  https://raw.githubusercontent.com/NVIDIA/NeMo-Curator/main/config/heuristic_filter_en.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4170 (4.1K) [text/plain]
Saving to: ‘./config/heuristic_filter_en.yaml’

./config/heuristic_ 100%[===================>]   4.07K  --.-KB/s    in 0s      

2025-01-05 21:53:27 (57.7 MB/s) - ‘./config/heuristic_filter_en.yaml’ saved [4170/4170]

nemo_output/remove_duplicate/part.0.parquet
Reading 1 files with blocksize='1gb' / files_per_partition=None


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:441: UserWarning: If underlying Parquet data does not have a consistent column order, reading with blocksize might fail. Please use files_per_partition approach instead.
  warnings.warn(


Writing to disk complete for 1 partition(s)


In [ ]:
from datasets import load_dataset

# Load the Parquet file as a Hugging Face dataset
parquet_dataset = load_dataset("parquet", data_files= HF_output_path + "/part.0.parquet")

# Access the "train" split of the dataset
train_split = parquet_dataset["train"]

# Visualize the dataset (e.g., first few rows)
print(train_split)

# View the values in the "text" column
print(train_split[:10]["text"])  # Show the first 10 values in the 'text' key

# Optional: Use the Dataset's `to_pandas` method for further visualization
cleaned_dff = train_split.to_pandas()
print(cleaned_dff.head())

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text'],
    num_rows: 61
})
['The New Orleans truck attack occurred on January 1, 2025, at 3:16 a.m. CST (UTC-6) on Bourbon Street, New Orleans, Louisiana, U.S. The attack targeted pedestrians and involved a vehicle-ramming attack, a shootout, and mass murder. The weapons used in the attack were a Ford F-150 Lightning and a .308 AR-10 semi-automatic rifle.', 'The 2025 New Orleans truck attack took place on January 1, 2025, at approximately 3:15 a.m. CST (UTC–6). A man drove a pickup truck into a crowd on Bourbon Street in New Orleans, Louisiana, United States. After crashing the truck, he exited the vehicle and engaged in a shootout with police officers before being fatally shot. The incident resulted in the deaths of fifteen people, including the suspect, and left at least 35 others injured. Among the injured were two police officers who sustained gunshot wounds. The attack happened during New Year celebrations in the city, which was set to host the col

In [ ]:
from nemo_curator import FuzzyDuplicates, FuzzyDuplicatesConfig


In [ ]:
added_id_output_path

'nemo_output/add_id'

In [ ]:

#Input
fuzzy_dedup_data_path = added_id_output_path
#Output
fuzzy_dedup_base_output_path = os.path.join(output_dir_nemo,"fuzzy_wrapper")
fuzzy_dedup_log_dir = os.path.join(fuzzy_dedup_base_output_path,'log')
fuzzy_dedup_cache_dir = os.path.join(fuzzy_dedup_base_output_path,'cache')
fuzzy_dedup_output_dir = os.path.join(fuzzy_dedup_base_output_path,'data')

#Relevant parameters
id_field = 'id'
text_field = 'text'
filetype = "parquet"

!mkdir -p {fuzzy_dedup_base_output_path}
!mkdir -p {fuzzy_dedup_log_dir}
!mkdir -p {fuzzy_dedup_cache_dir}
!mkdir -p {fuzzy_dedup_output_dir}

In [ ]:
fuzzy_dedup_data_path

'nemo_output/add_id'

In [ ]:
import dask_cudf
import dask


with dask.config.set({"dataframe.backend": 'cudf'}):
        # input_dataset = DocumentDataset.read_parquet("added_id_output_path", backend='cudf')

        input_dataset = DocumentDataset.read_parquet(added_id_output_path, backend='cudf')

        fuzzy_dedup_config = FuzzyDuplicatesConfig(
            cache_dir=fuzzy_dedup_cache_dir,
            id_field=id_field,
            text_field=text_field,
            seed=42, #Use the seed set in Minhash section for consistency
            char_ngrams=5,
            num_buckets=20,
            hashes_per_bucket=13,
            use_64_bit_hash=False,
            buckets_per_shuffle=2,
            false_positive_check=True,
            num_anchors=2,
            jaccard_threshold=0.8,
        )
        fuzzy_dup = FuzzyDuplicates(logger=fuzzy_dedup_log_dir, config=fuzzy_dedup_config)
        duplicates = fuzzy_dup(dataset=input_dataset)

        duplicates.to_parquet(fuzzy_dedup_output_dir, write_to_filename=False)


Reading 1 files with blocksize='1gb' / files_per_partition=None


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:436: UserWarning: If underlying Parquet data does not have consistent schema, reading with blocksize will fail. Please update underlying RAPIDS package to version 25.02 or higher, or use files_per_partition approach instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/nemo_curator/modules/config.py:91: UserWarning: Identifying false positives during the Minhash deduplication is computationally expensive. For improved performance consider setting this to False
  warnings.warn(
INFO:FuzzyDuplicates:Starting execution for Minhashes
INFO:FuzzyDuplicates:Time taken for Minhash signature computation = 0.08594417572021484s and output written at nemo_output/fuzzy_wrapper/cache/_minhashes.parquet


Stage1: Starting Minhash + LSH computation


INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_0', '_bucket_1']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_2', '_bucket_3']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_4', '_bucket_5']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_6', '_bucket_7']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_8', '_bucket_9']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_10', '_bucket_11']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_12', '_bucket_13']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_14', '_bucket_15']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_16', '_bucket_17']
INFO:FuzzyDuplicates:Wrote data for buckets: ['_bucket_18', '_bucket_19']
INFO:FuzzyDuplicates:Time taken for LSH = 7.381689786911011s and output written at nemo_output/fuzzy_wrapper/cache/_buckets.parquet


Stage1: Minhash + LSH complete!
Stage2 (False Positive Check): Starting Map_Buckets


...

Text-df partition  1/1 completed in 0.2392582893371582
Bucket partition  1/1 completed in 0.24610114097595215


100%|██████████| 1/1 [00:00<00:00,  4.00it/s]
INFO:FuzzyDuplicates:Time taken for Shuffle = 0.26691341400146484s and output written at nemo_output/fuzzy_wrapper/cache/shuffled_docs.parquet
INFO:FuzzyDuplicates:Time taken for Jaccard Similarity = 0.07744050025939941s and output written at nemo_output/fuzzy_wrapper/cache/jaccard_similarity_results.parquet


Stage3 (False Postive Check): Shuffle docs complete!
Stage4 (False Postive Check): Jaccard Similarity in Buckets
Stage4 (False Postive Check): Jaccard Similarity in Buckets Complete!
Stage5: Connected Components across buckets


INFO:FuzzyDuplicates:Time taken for Dedup Parsed Id = 0.2141261100769043s and output written at nemo_output/fuzzy_wrapper/cache/dedup_parsed_id.parquet
INFO:FuzzyDuplicates:Time taken for merge and write = 0.15172696113586426s and output written at nemo_output/fuzzy_wrapper/cache/encoded_jaccard_pair/
INFO:FuzzyDuplicates:Time taken for Encoding Jaccard Pairs = 0.16474032402038574s and output written at nemo_output/fuzzy_wrapper/cache/encoded_jaccard_pair/
INFO:FuzzyDuplicates:Time taken for Dedup Encoding Jaccard Pairs = 0.13334345817565918s and output written at nemo_output/fuzzy_wrapper/cache/final_dedup_encoded_jaccard_pair.parquet
INFO:distributed.worker:Run out-of-band function '_func_set_scheduler_as_nccl_root'
/usr/local/lib/python3.10/dist-packages/cudf/core/reshape.py:383: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavio

Stage5: Connected Components across buckets complete!
Writing to disk complete for 1 partition(s)


In [ ]:
fuzzy_dedup_output_dir

'nemo_output/fuzzy_wrapper/data'

In [ ]:
# result

,id,text
npartitions=1,,
,object,object
,...,...


In [ ]:
input_dataset = DocumentDataset.read_parquet(added_id_output_path, backend="cudf")
result = input_dataset.df

Reading 1 files with blocksize='1gb' / files_per_partition=None


/usr/local/lib/python3.10/dist-packages/nemo_curator/utils/distributed_utils.py:436: UserWarning: If underlying Parquet data does not have consistent schema, reading with blocksize will fail. Please update underlying RAPIDS package to version 25.02 or higher, or use files_per_partition approach instead.
  warnings.warn(


In [ ]:
#Loads result from fuzzy dedup wrapper
fuzzy_duplicates = pd.read_parquet(fuzzy_dedup_output_dir)

#Generate list of near duplicate document ID
fuzzy_docs_to_remove = fuzzy_duplicates[fuzzy_duplicates.duplicated(subset=['group'], keep='first')]

In [ ]:
fuzzy_docs_to_remove

,id,group
1,VI_-0000000031,1


In [ ]:
#Remove near duplicates
result = result[~result["id"].isin(fuzzy_docs_to_remove["id"])]

#Save final result to local
result.to_parquet("nemo_output/deduped", write_to_filename=True)

In [ ]:
from datasets import load_dataset

# Load the Parquet file as a Hugging Face dataset
parquet_dataset = load_dataset("parquet", data_files= "nemo_output/deduped" + "/part.0.parquet")

# Access the "train" split of the dataset
train_split = parquet_dataset["train"]

# Visualize the dataset (e.g., first few rows)
print(train_split)

# View the values in the "text" column
print(train_split[:10]["text"])  # Show the first 10 values in the 'text' key

# Optional: Use the Dataset's `to_pandas` method for further visualization
cleaned_dff = train_split.to_pandas()
print(cleaned_dff.head())

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['__null_dask_index__', 'id', 'text'],
    num_rows: 97
})
['The New Orleans truck attack occurred on January 1, 2025, at 3:16 a.m. CST (UTC-6) on Bourbon Street, New Orleans, Louisiana, U.S. The attack targeted pedestrians and involved a vehicle-ramming attack, a shootout, and mass murder. The weapons used in the attack were a Ford F-150 Lightning and a .308 AR-10 semi-automatic rifle.', 'The New Orleans truck attack resulted in 15 deaths, including the suspect, and left 35 people injured. The motive behind the attack is currently under investigation.', 'The 2025 New Orleans truck attack took place on January 1, 2025, at approximately 3:15 a.m. CST (UTC–6). A man drove a pickup truck into a crowd on Bourbon Street in New Orleans, Louisiana, United States. After crashing the truck, he exited the vehicle and engaged in a shootout with police officers before being fatally shot. The incident resulted in the deaths of fifteen people, including the suspect, and left 

In [ ]:
cleaned_dff

,__null_dask_index__,id,text
0,0,VI_-0000000000,The New Orleans truck attack occurred on Janua...
1,1,VI_-0000000001,The New Orleans truck attack resulted in 15 de...
2,2,VI_-0000000002,The 2025 New Orleans truck attack took place o...
3,3,VI_-0000000003,The attack took place during New Year celebrat...
4,4,VI_-0000000004,The Federal Bureau of Investigation (FBI) iden...
...,...,...,...
92,93,VI_-0000000093,"Following the crash, Jeju Air recorded an incr..."
93,94,VI_-0000000094,"On 30 December, authorities ordered the inspec..."
94,95,VI_-0000000095,The United States deployed investigators from ...
95,96,VI_-0000000096,"GE Aerospace, which produced the engines on th..."


In [ ]:
import shutil
import os

# Specify the directory to delete
directory_to_delete = "/content/nemo_output/fuzzy_wrapper"

# Check if the directory exists
if os.path.exists(directory_to_delete):
    # Delete the directory
    shutil.rmtree(directory_to_delete)
    print(f"Directory '{directory_to_delete}' has been deleted.")
else:
    print(f"Directory '{directory_to_delete}' does not exist.")


Directory '/content/nemo_output/fuzzy_wrapper' has been deleted.
